### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev sentence pairs, and unlabeled test sentence pairs, into lists.

In [1]:
import csv

In [2]:
train, dev, test = [], [], []

In [3]:
with open('./data/pnli_train.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        train.append(x)
print (len(train))
print (train[:3])

5983
[['Sometimes do exercise.', 'A person typically desire healthy life.', '1'], ['Who eats junk foods.', 'A person typically desire healthy life.', '0'], ['A person is sick.', 'A person typically desire healthy life.', '1']]


In [4]:
with open('./data/pnli_dev.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[2] will be the label (0 or 1). x[0] and x[1] will be the sentence pairs.
        dev.append(x)
print (len(dev))
print (dev[:3])

1055
[['A person is looking for accuracy.', 'A person typically desires accurate results.', '1'], ['A person does not care for accuracy.', 'A person typically desires accurate results.', '0'], ['The person double checks their data.', 'A person typically desires accurate results.', '1']]


In [6]:
with open('./data/pnli_test_unlabeled.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for x in csvreader:
        # x[0] and x[1] will be the sentence pairs.
        test.append(x)
print (len(test))
print (test[:3])

4850
[['The people want to have a romantic and pleasant feel.', 'People typically does desire to smell violets.'], ['The contract is to buy products from you.', 'Getting contract typically cause to make money or spend money.'], ['Train station is closed.', 'Line can typically be used to move train along tracks.']]


### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [13]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []

### Training Code

In [ ]:
import datasets
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, DataCollatorWithPadding, Trainer
from transformers import AutoConfig

In [ ]:
data_files = {
    'train': 'data/pnli_train.csv',
    'validation': 'data/pnli_dev.csv',
}

In [ ]:
raw_datasets = datasets.load_dataset('csv', data_files=data_files, column_names=['sentence1', 'sentence2', 'labels'])

In [ ]:
checkpoint = 'facebook/bart-large-mnli'
checkpoint_folder = 'facebook-bart-large-mnli'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    'checkpoints/' + checkpoint_folder + 'checkpoint',
    evaluation_strategy='steps',
    num_train_epochs=10,
    eval_steps=500,
    warmup_steps=200,
    metric_for_best_model='accuracy',
    load_best_model_at_end=True
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
def compute_metrics(eval_preds):
    print(type(eval_preds))
    logits, labels = eval_preds
    logits = logits[0]
    predictions = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('models/' + checkpoint_folder)

### Prediction Code

In [ ]:
import torch
import datasets
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
local_model_path = 'models/facebook-bart-large-mnli/'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(local_model_path, local_files_only=True, num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
raw_datasets = datasets.load_dataset('csv', data_files={'test':'data/pnli_test_unlabeled.csv'}, column_names=['sentence1', 'sentence2', 'labels'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)

In [ ]:
results = []
for i in range(0, len(raw_datasets['test']["sentence1"]), 100):
    sentence1 = raw_datasets['test']["sentence1"][i:i+100]
    sentence2 = raw_datasets['test']['sentence2'][i:i+100]
    tokenized_datasets = tokenizer(sentence1, sentence2, padding=True, 
                               truncation=True, return_tensors='pt')
    pred = model(**tokenized_datasets)
    pred = np.argmax(pred.logits.detach().numpy(), axis=1)
    results.extend(pred)

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [ ]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 4850)

In [ ]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [ ]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')